In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/congressional_voting_dataset.csv')

## Wstępna eksploracja

Z informacji podanych na https://www.apispreadsheets.com/datasets/121 dowiadujemy się, że tabela przedstawiaa informację o głosach kongresmenów w 16 różnych, kluczowych głosowaniach. Oznaczenie:
- y - kongresmen oddał głos za 
- n - głos przeciw
- ? - nie zagłosował, wstrzymał się


In [ ]:
df.head()

In [ ]:
df.info()

Tabela nie zawiera, żadnych braków informacji. Więc od razu przechodzimy do ekploracji danych. Jeszcze przed tym sprawdzając czy tabela nie zawiera, żadnych błędów.

In [ ]:
df.apply(pd.unique, axis=0)

In [ ]:
change_dict = {"y": 1, "n": -1, "?": 0, "republican": 0, "democrat": 1 }
df.replace(change_dict, inplace=True)

Zamieniam dane typu str na numeryczne aby łatwiej je było analizować.
- -1 to głos przeciw
- 0 powstrzymanie się od głosu
- 1 głos za


W ostatniej kolumnie:
- 0 republikanin
- 1 demokrata

In [ ]:
df.hist(figsize=(12,12))
plt.show

Co można zauważyć to, że w naszych danych jest o ok. 100 więcej demokratów. Poza dwoma głosowaniami w których była duża liczba wstrzymań od głosów to głosy rozkładają się mniej więcej równo co może wskazywać na głosowanie według partyjnych podziałów.

Zobaczmy jak rozdzielają się te głosowania które odznaczają się dużą liczbą głosów powstrzymanych. Ciekawe jest to, że głosy nie rozkładają się jasno wobec podziału na partię. A w sprawie pierwszego przedstwionego niżej głosowania wygląda jaby przynależność partyjna nie miała, żadnego znaczenia. Było to na tyle zaskakujące, że aż sprawdziłem dokładne wartości.

In [ ]:
sns.countplot(x=df.columns[1],hue=df.columns[-1], data=df)
plt.legend(["republican", "democrat"])
plt.show()

In [ ]:
gr = df.groupby(['water_project_cost_sharing', 'political_party'])['water_project_cost_sharing'].count()
gr

In [ ]:
sns.countplot(x=df.columns[-2],hue=df.columns[-1], data=df).set_title("Rozkłąd głosów z podziałem na partię")
plt.legend(["republican", "democrat"])
plt.show()

Sprawdzam czy istnieją jakieś *outliery* które w tym wypdaku są np powstrzymywaniem się od głosu za każdym razem. Mamy jednego takiego kongresmena

In [ ]:
def ff(x):
  return all(x == 0) or all(x == -1) or all(x == 1)
df[df.drop('political_party', axis=1).apply(func=ff, axis=1)]

In [ ]:
dfa = abs(df)
dfaa = np.apply_along_axis(sum, 1, dfa)
sns.countplot(dfaa)
plt.title("Ilość głosujących wegług liczby zagłosowań")

In [ ]:
print(dfaa[dfaa < 11])


## Rozkład głosów poszczególnych kongresmenów
Wykres ten miał za zadanie sprawdzić 

In [ ]:
vot = df.drop('political_party', axis=1).apply(func=sum, axis=1)
vot = pd.DataFrame(data=(vot, df['political_party'])).transpose()
vot.columns.values[[0,1]] = ('vots', 'party')
vot.sort_values(by='vots', inplace=True)
vot["index"] = pd.Series(range(0, len(vot)), index=vot.index)

In [ ]:

plt.figure(figsize=(15, 10))
sns.scatterplot(x=vot.index, y='vots', hue='party', data=vot)
#plt.legend(['republican', 'democrat'])
plt.show()

In [ ]:
df.iloc[[256, 47, 94]]

In [ ]:
mask = np.zeros_like(df.corr())
mask[np.triu_indices_from(mask)] = True
with sns.axes_style('ticks'):
    f, ax = plt.subplots(figsize=(12,8))
    ax = sns.heatmap(df.corr(), mask=mask, vmax=.3,annot=True,fmt=".0%",linewidth=0.5,square=False,cmap='Purples')
    plt.title('Macierz korelacji')

In [ ]:
f, axes = plt.subplots(4,4,figsize=(16,16))

for i in range(0, len(df.columns)):
  sns.countplot(x=df.columns[i],hue=df.columns[-1], data=df, ax=axes[i//4, i%4])
plt.legend('republican', 'democrat')
plt.show()

In [ ]:
def grr(x, df):
  return df.groupby([x])[x].count()

In [ ]:
## Ile procent parti głosowało na każdy z pomysłów
def grra(df, k):
  dfd = df[df.political_party == k].drop('political_party', axis=1)
  l = len(dfd['handicapped_infants'])
  dfdg = pd.DataFrame()
  for i in dfd.columns:
    dfdg[i] = 100*grr(i, dfd)/l
  return dfdg

In [ ]:
df_rep = grra(df, 0)
df_dem = grra(df,0)
df_rep.index